In [2]:
# import json

# CREDENTIALS_FILENAME = 'twitter_credentials.json'
# jf = open(CREDENTIALS_FILENAME)
# creds = json.load(jf)
# jf.close()

# client = Twython(creds['CONSUMER_KEY'],
#                   creds['CONSUMER_SECRET'],
#                   creds['ACCESS_TOKEN'],
#                   creds['ACCESS_TOKEN_SECRET'])


#Astra Project?
#Search API
import json

# Enter your keys/secrets as strings in the following fields
creds = {}
creds['CONSUMER_KEY'] = ""
creds['CONSUMER_SECRET'] = ""
creds['ACCESS_TOKEN'] = ""
creds['ACCESS_SECRET'] = ""

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(creds, file)

In [1]:
from twython import Twython
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])#creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'] WORKED!

# Create our query
query = {'q': 'learn python',
        'result_type': 'popular',
        'count': 10,
        'lang': 'en',
        }

In [2]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': []}
for status in python_tweets.search(**query)['statuses']:
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['favorite_count'].append(status['favorite_count'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(5)

,user,date,text,favorite_count
1,NASAEarthData,Thu Oct 24 14:26:08 +0000 2019,Check out this #Python tutorial to learn how t...,107
2,KirkDBorne,Fri Oct 25 23:47:35 +0000 2019,[FREE e-Book] Learn Classification &amp; Regre...,68
0,xdadevelopers,Tue Oct 29 14:37:16 +0000 2019,Learn Python Programming From Scratch With Thi...,64


In [3]:
from twython import TwythonStreamer
import csv

# Filter out unwanted data
def process_tweet(tweet):
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d
    
    
# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()
        
    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [4]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'], 
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='python')# ran out of limit..420 b'Exceeded connection limit for user\r\n'

KeyboardInterrupt: 

In [5]:
import pandas as pd
tweets = pd.read_csv("saved_tweets.csv")
tweets.head()

,[],Better Python print statement:\n\nname = 'Peter'\nage = 23\nprint(f'{name} is {age} years old')\n\nhttps://t.co/4uZyiW6HiE,andriyko,"Ottawa, ON, Canada"
0,"['devops', 'share', 'python', 'falconframework...",RT @periket2000: 2019-10-29 06:00:01 #devops S...,boina92,Paris
1,"['BigData', 'Analytics', 'DataScience', 'IoT',...",RT @gp_pulipaka: A Gentle Introduction to Cond...,ArkangelScrap,France
2,"['TensorFlow', 'BigData', 'Analytics', 'DataSc...",RT @gp_pulipaka: TensorFlow 2.0: A Concise ML ...,ArkangelScrap,France
3,"['BigData', 'Analytics', 'MachineLearning', 'D...",RT @gp_pulipaka: Rethinking Atrous Convolution...,ArkangelScrap,France
4,"['Cancer', 'Keras', 'BigData', 'Analytics', 'D...",RT @gp_pulipaka: Breast #Cancer Classification...,ArkangelScrap,France


In [6]:
from collections import Counter
import ast

tweets = pd.read_csv("saved_tweets.csv")

# Extract hashtags and put them in a list
list_hashtag_strings = (entry for entry in tweets.hashtags)
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)
counter_hashtags.most_common(20)

AttributeError: 'DataFrame' object has no attribute 'hashtags'

In [13]:
import sys
!{sys.executable} -m pip install aomixcloud

  Could not find a version that satisfies the requirement aomixcloud (from versions: )
No matching distribution found for aomixcloud
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
